# Introduction to JAX + Flax

This notebook is meant to give a short introduction to JAX, including writing and training your own neural network with [Flax](https://flax.readthedocs.io/en/latest/). 

The reason to learn JAX, is this framework extremely fast. For instance, a small GoogleNet on CIFAR10, can be trained in JAX 3x faster than in PyTorch with a similar setup. For large language model like GPT, JAX will be a big winner. JAX enables the speedup by compiling functions and numerical programs for accelerators (GPU/TPU) `just in time` with backend [XLA](https://github.com/openxla/xla), finding the optimal utilization of the hardwor. PyTorch is built on dynamic computation graphs which can not anticipate the next operations before the user calls them. That's way JAX with static graph can pre-optimize the execution path. 


In order to efficiently compile programs just-in-time in JAX, the functions need to be written with certain constraints:
1. The functions are not allowed to have side-effects.
2. stochastic operations such as `torch.ran(..)` change the global state of pseudo random number generators, which is not allowed.
3. Since the JAX compiles the functions based on anticipated shape of their input. It can not handle the dynamo shape input.

Beside, JAX can combine it parts from PyTorch (e.g., data loading) and TensorFlow (e.g., logging in TensorBoard).

Future, we use Flax as a neural network library in JAX, dn Optax to implement common deep learning optimizers. More on them later in the notebook.

In [1]:
import os
import math
import numpy as np
import time

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.colors import to_rgba
import seaborn as sns
sns.set_theme()

## Progress bar
from tqdm.auto import tqdm

## JAX as NumPy on accelerators
this basic API strongly resembles the one of NumPy, and even has the same name in JAX (`jax.numpy`). So, for now, let’s think of JAX as NumPy that runs on accelerators. As a first step, let’s import JAX and its NumPy API:

In [2]:
import jax
import jax.numpy as jnp
print("Using jax", jax.__version__)

Using jax 0.5.0


The NumPy API of JAX is usually imported as `jnp`, to keep a resemblance to NumPy’s import as `np`

### Device Arrays

As a first test, let’s create some arbitrary arrays like we would do in NumPy. For instance, let’s create an array of zeros with shape `[2,5]`:

In [3]:
a = jnp.zeros((2, 5), dtype=jnp.float32)
print(a)

W0000 00:00:1737152282.679843 26455892 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1737152282.694085 26455892 service.cc:145] XLA service 0x11fbb14d0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737152282.694107 26455892 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1737152282.696009 26455892 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1737152282.696025 26455892 mps_client.cc:384] XLA backend will use up to 28990554112 bytes on device 0 for SimpleAllocator.


Metal device set to: Apple M3 Pro

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


Similarly, we can create an array with values of 0 to 5 by using `arange`:

In [4]:
b = jnp.arange(6)
print(b)

[0 1 2 3 4 5]


You might now wonder whether the arrays `a` and `b` are simply NumPy arrays. To check that, let’s print out the class of `b`:

In [5]:
b.__class__

jaxlib.xla_extension.ArrayImpl

Instead of a simple NumPy array, it shows the type `DeviceArray` which is what JAX uses to represent arrays. In contrast to NumPy, JAX can execute the same code on different backends – CPU, GPU and TPU. A `DeviceArray` therefore represents an array which is on one of the backends. Similar to PyTorch, we can check the device of an array by calling `.devices()`:

In [8]:
b.devices()

{METAL(id=0)}

As you can see, the array b is already natively on a `METAL` (mac GPU) although we did not specify this explicitly as you would do in PyTorch (on Colab, remember to select a GPU in your runtime environment). In order to change the device of an array, we can use jax.device_get:

In [10]:
b_cpu = jax.device_get(b)
print(b_cpu.__class__)

<class 'numpy.ndarray'>


Unsurprisingly, a simple CPU-based array is nothing else than a NumPy array, which allows for a simple conversion between the two frameworks! To explicitly push a NumPy array to the accelerator, you can use `jax.device_put`:

In [11]:
b_gpu = jax.device_put(b_cpu)
print(b_gpu.__class__, b_gpu.devices())

<class 'jaxlib.xla_extension.ArrayImpl'> {METAL(id=0)}


Nicely enough, JAX will handle any device clash itself when you try to perform operations on a NumPy array and a DeviceArray by modeling the output as DeviceArray again:


In [13]:
result = b_cpu + b_gpu
print(result)
print(result.__class__)

[ 0  2  4  6  8 10]
<class 'jaxlib.xla_extension.ArrayImpl'>


Finally, we can also print all our available devices using jax.devices():

In [14]:
jax.devices()

[METAL(id=0)]

A technical detail of running operations on DeviceArrays is that when a JAX function is called, the corresponding operation takes place asynchronously on the accelerator when possible. For instance, if we call out = jnp.matmul(b, b), JAX first returns a placeholder array for out which may not be filled with the values as soon as the function calls finishes. This way, Python will not block the execution of follow-up statements, but instead only does it whenever we strictly need the value of out, for instance for printing or putting it on CPU. PyTorch uses a very similar principle to allow asynchronous computation. For more details, see [JAX - Asynchronous Dispatch](https://jax.readthedocs.io/en/latest/async_dispatch.html).


### Immutable tensors

When we would like to change a NumPy array in-place, like replacing the first element of `b` with `1` instead of 0, we could simply write `b[0]=1`. However, in JAX, this is not possible. A `DeviceArray` object is immutable, which means that no in-place operations are possible. The reason for this goes back to our discussion in the introduction: JAX requires programs to be “pure” functions, i.e. no effects on variables outside of the function are allowed. Allowing in-place operations of variables would make the program analysis for JAX’s just-in-time compilation difficult. Instead, we can use the expression `b.at[0].set(1)` which, analogous to the in-place operation, returns a new array which is identical to `b`, except that its value at the first position is 1. Let’s try that out below:

In [15]:
b_new = b.at[0].set(1)
print('Original array:', b)
print('Changed array:', b_new)

Original array: [0 1 2 3 4 5]
Changed array: [1 1 2 3 4 5]


However, we said that JAX is very efficient. Isn’t creating a new array in this case the opposite? While it is indeed less efficient, it can made much more efficient with JAX’s just-in-time compilation. The compiler can recognize unnecessary array duplications, and replace them with in-place operations again. More on the just-in-time compilation later!